<a href="https://colab.research.google.com/github/SRARNAB7/HDS_5230_07_Arnab/blob/main/Week_3_Assignment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [68]:
import pandas as pd
import numpy as np
from math import *

# **Read in the data**

In [69]:
#del df
df = pd.read_csv('clinics.csv', delimiter='|')
df.head()

,bizID,bizCat,bizCatSub,bizName,bizAddr,bizCity,bizState,bizZip,bizPhone,bizFax,...,bizURL,locAreaCode,locFIPS,locTimeZone,locDST,locLat,locLong,locMSA,locPMSA,locCounty
0,1,Clinics,Clinics,Hino Ronald H MD,98-151 Pali Momi Street Suite 142,Aiea,HI,96701,(808)487-2477,NaN,...,NaN,808,15003,PST-2,N,21.3980,-157.8981,3320.0,NaN,Honolulu
1,2,Clinics,Clinics,Farmer Joesph F Md,1225 Breckenridge Drive,Little Rock,AR,72205,(501)225-2594,NaN,...,NaN,501,5119,CST,Y,34.7495,-92.3533,4400.0,NaN,Pulaski
2,3,Clinics,Clinics & Medical Centers,Najjar Fadi Md,1155 West Linda Avenue Suite B,Hermiston,OR,97838,(541)289-1122,NaN,...,NaN,541,41059,PST,Y,45.8456,-119.2817,NaN,NaN,Umatilla
3,4,Clinics,Clinics & Medical Centers,Kittson Memorial Upper Level Nursing Home,1010 South Birch Avenue,Hallock,MN,56728,(218)843-2525,NaN,...,NaN,218,27069,CST,Y,48.7954,-97.0090,NaN,NaN,Kittson
4,5,Clinics,Clinics & Medical Centers,Thompson Robert B Md,100 North Eagle Creek Drive,Lexington,KY,40509,(859)258-4000,NaN,...,www.lexingtonclinic.com,859,21067,EST,Y,37.9935,-84.3712,4280.0,NaN,Fayette


In [70]:
df.columns = df.columns.str.strip()

# **Haversine definition**

In [71]:
def haversine(lat1, lon1, lat2, lon2):
    miles_constant = 3959
    lat1, lon1, lat2, lon2 = map(np.deg2rad, [lat1, lon1, lat2, lon2])
    dlat = lat2 - lat1
    dlon = lon2 - lon1
    a = np.sin(dlat/2)**2 + np.cos(lat1) * np.cos(lat2) * np.sin(dlon/2)**2
    c = 2 * np.arcsin(np.sqrt(a))
    mi = miles_constant * c
    return mi

In [72]:
%%timeit
distances = []
for i in range(len(df)):
    distances.append(haversine(40.671, -73.985, df.loc[i, 'locLat'], df.loc[i, 'locLong']))
df['distance'] = distances


917 µs ± 10.1 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


In [75]:
%%timeit
distances = []
for i in range(len(df)):
    distances.append(haversine(40.671, -73.985, df['locLat'], df['locLong']))
df['distance'] = distances

34.5 ms ± 4.02 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [76]:
%%timeit
distances = []
for i in range(len(df)):
    distances.append(haversine(40.671, -73.985, df['locLat'][i], df['locLong'][i]))  # ✅ Correct
df['distance'] = distances

830 µs ± 198 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


# **Iterrows Haversine**

In [ ]:
%%timeit
# Haversine applied on rows via iteration
haversine_series = []
for index, row in df.iterrows():
    haversine_series.append(haversine(40.671, -73.985,\
                                      row['locLat'], row['locLong']))
df['distance'] = haversine_series

2.08 ms ± 186 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


# **Apply Haversine on rows**

In [ ]:
%timeit df['distance'] =\
df.apply(lambda row: haversine(40.671, -73.985,\
                               row['locLat'], row['locLong']), axis=1)

1 ms ± 22.1 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


In [ ]:
!pip install line_profiler
%load_ext line_profiler
import line_profiler


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 750.2/750.2 kB 12.4 MB/s eta 0:00:00


In [ ]:
# Haversine applied on rows
%lprun -f haversine \
df.apply(lambda row: haversine(40.671, -73.985,\
                               row['locLat'], row['locLong']), axis=1)

# **Vectorized implementation of Haversine applied on Pandas series**

**Timing vectorized implementation**

In [ ]:
# Vectorized implementation of Haversine applied on Pandas series
%timeit df['distance'] = haversine(40.671, -73.985,\
                                   df['locLat'], df['locLong'])

1.36 ms ± 205 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


**Profiling vectorized implementation**


In [ ]:
# Vectorized implementation profile
%lprun -f haversine haversine(40.671, -73.985,\
                              df['locLat'], df['locLong'])


# **Vectorized implementation of Haversine applied on NumPy arrays**

**Timing vectorized implementation**

In [ ]:
# Vectorized implementation of Haversine applied on NumPy arrays
%timeit df['distance'] = haversine(40.671, -73.985,\
                         df['locLat'].values, df['locLong'].values)

164 µs ± 7.87 µs per loop (mean ± std. dev. of 7 runs, 10000 loops each)


In [ ]:
%%timeit
# Convert pandas arrays to NumPy ndarrays
np_lat = df['locLat'].values
np_lon = df['locLong'].values

6.19 µs ± 50.5 ns per loop (mean ± std. dev. of 7 runs, 100000 loops each)


In [ ]:
# Vectorized implementation of Haversine applied on NumPy arrays
%timeit df['distance'] = haversine(40.671, -73.985,\
                         df['locLat'].values, df['locLong'].values)

170 µs ± 8.19 µs per loop (mean ± std. dev. of 7 runs, 10000 loops each)


**Profiling vectorized implementation**


In [ ]:
%lprun -f haversine df['distance'] = haversine(40.671, -73.985,\
                        df['locLat'].values, df['locLong'].values)